In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import sys

sys.path.insert(0, str(Path.cwd().parent.parent.parent))

from modules._import_helper import safe_import_from

set_seed = safe_import_from('00_repo_standards.src.mlphys_core.seeding', 'set_seed')
condition_number = safe_import_from('01_numerical_toolbox.src.linear_algebra', 'condition_number')

set_seed(42)
REPORTS_DIR = Path('../reports')
REPORTS_DIR.mkdir(parents=True, exist_ok=True)

print("✓ Imports successful")

---

## 1. Intuition: Why Ridge?

**Two complementary views:**

**Machine Learning View (Regularization):**
- Prevents overfitting by penalizing large weights
- Bias-variance tradeoff: small λ → low bias, high variance; large λ → high bias, low variance
- Shrinks coefficients toward zero

**Numerical Analysis View (Stabilization):**
- Fixes ill-conditioned design matrices (X^T X nearly singular)
- Makes matrix inversion numerically stable
- Reduces condition number κ(X^T X + λI) ≪ κ(X^T X)

**Key Insight**: These are the same phenomenon! Ill-conditioning → unstable coefficients → high variance → overfitting.

---

## 2. Minimal Math

### Ordinary Least Squares (OLS)
Minimize: $\|\mathbf{y} - X\boldsymbol{\beta}\|^2$

Solution (normal equations): 
$$
\hat{\boldsymbol{\beta}}_{\text{OLS}} = (X^T X)^{-1} X^T \mathbf{y}
$$

**Problem**: If $X^T X$ is ill-conditioned (nearly singular), $\hat{\boldsymbol{\beta}}$ becomes:
- Highly sensitive to noise in y
- Numerically unstable (rounding errors amplified)
- Large in magnitude (overfitting)

### Ridge Regression (L2 Regularization)
Minimize: $\|\mathbf{y} - X\boldsymbol{\beta}\|^2 + \lambda \|\boldsymbol{\beta}\|^2$

Solution:
$$
\hat{\boldsymbol{\beta}}_{\text{Ridge}} = (X^T X + \lambda I)^{-1} X^T \mathbf{y}
$$

**Key**: Adding $\lambda I$ (identity matrix) to $X^T X$:
1. Shifts all eigenvalues by λ: $\text{eig}(X^T X + \lambda I) = \text{eig}(X^T X) + \lambda$
2. Increases smallest eigenvalue most → reduces condition number
3. Makes inversion stable: $\kappa(X^T X + \lambda I) \approx \frac{\lambda_{\max} + \lambda}{\lambda}$ for small $\lambda_{\min}$

---

## 3. Implementation: OLS and Ridge from Scratch

In [ ]:
def fit_ols(X, y, add_intercept=True):
    """Fit OLS via normal equations."""
    if add_intercept:
        X = np.column_stack([np.ones(len(X)), X])
    
    # β = (X^T X)^{-1} X^T y
    beta = np.linalg.solve(X.T @ X, X.T @ y)
    return beta

def fit_ridge(X, y, lambda_reg=1.0, add_intercept=True):
    """Fit Ridge regression via modified normal equations."""
    if add_intercept:
        X = np.column_stack([np.ones(len(X)), X])
    
    n_features = X.shape[1]
    # Don't penalize intercept
    penalty_matrix = np.eye(n_features)
    penalty_matrix[0, 0] = 0  # No penalty on intercept
    
    # β = (X^T X + λI)^{-1} X^T y
    beta = np.linalg.solve(X.T @ X + lambda_reg * penalty_matrix, X.T @ y)
    return beta

def predict(X, beta, add_intercept=True):
    """Generate predictions."""
    if add_intercept:
        X = np.column_stack([np.ones(len(X)), X])
    return X @ beta

def mse(y_true, y_pred):
    """Mean squared error."""
    return np.mean((y_true - y_pred)**2)

print("✓ Functions defined")

---

## 4. Experiment: Ill-Conditioned Data with Correlated Features

In [ ]:
# Generate ill-conditioned data
np.random.seed(42)
n_samples = 100
n_features = 5

# Create highly correlated features
# Feature 1 is base, others are linear combinations + small noise
X = np.random.randn(n_samples, 1)
for i in range(n_features - 1):
    # Each new feature = previous feature + tiny noise
    X = np.column_stack([X, X[:, -1] + 0.1 * np.random.randn(n_samples)])

# True relationship: y = 3*x1 - 2*x2 + 1*x3 + 0*x4 + 0*x5 + noise
true_beta = np.array([3, -2, 1, 0, 0])
y = X @ true_beta + 0.5 * np.random.randn(n_samples)

# Standardize
X_mean, X_std = X.mean(axis=0), X.std(axis=0)
X_scaled = (X - X_mean) / X_std

# Check conditioning
XtX = X_scaled.T @ X_scaled
kappa = condition_number(XtX)
print(f"Condition number of X^T X: {kappa:.1f}")
print(f"This is {'well' if kappa < 10 else 'ill'}-conditioned!")
print(f"\nFeature correlations:")
corr_matrix = np.corrcoef(X.T)
for i in range(min(3, n_features)):
    for j in range(i+1, min(4, n_features)):
        print(f"  Feature {i+1} vs {j+1}: {corr_matrix[i,j]:.3f}")

**Observation**: Features are highly correlated (ρ ≈ 0.9+) → X^T X is ill-conditioned!

---

## 5. Coefficient Instability: OLS vs Ridge

In [ ]:
# Split data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.3, random_state=42
)

# Fit OLS
beta_ols = fit_ols(X_train, y_train, add_intercept=True)
y_pred_ols_train = predict(X_train, beta_ols)
y_pred_ols_test = predict(X_test, beta_ols)

print("OLS Coefficients (excluding intercept):")
print(beta_ols[1:])  # Skip intercept
print(f"\nMagnitude: ||β||₂ = {np.linalg.norm(beta_ols[1:]):.2f}")
print(f"Train MSE: {mse(y_train, y_pred_ols_train):.3f}")
print(f"Test MSE:  {mse(y_test, y_pred_ols_test):.3f}")

# Fit Ridge with λ = 1.0
beta_ridge = fit_ridge(X_train, y_train, lambda_reg=1.0, add_intercept=True)
y_pred_ridge_train = predict(X_train, beta_ridge)
y_pred_ridge_test = predict(X_test, beta_ridge)

print("\nRidge Coefficients (λ=1.0):")
print(beta_ridge[1:])
print(f"\nMagnitude: ||β||₂ = {np.linalg.norm(beta_ridge[1:]):.2f}")
print(f"Train MSE: {mse(y_train, y_pred_ridge_train):.3f}")
print(f"Test MSE:  {mse(y_test, y_pred_ridge_test):.3f}")

print(f"\n→ Coefficient norm reduced by {100*(1 - np.linalg.norm(beta_ridge[1:])/np.linalg.norm(beta_ols[1:])):.0f}%")
print(f"→ Test error {'improved' if mse(y_test, y_pred_ridge_test) < mse(y_test, y_pred_ols_test) else 'degraded'}")

---

## 6. Sensitivity to Noise (Monte Carlo)

In [ ]:
# Test stability across multiple noisy realizations
n_trials = 50
betas_ols = []
betas_ridge = []

for seed in range(n_trials):
    np.random.seed(seed)
    # Add different noise to same X
    y_noisy = X_train @ true_beta[:n_features] + 0.5 * np.random.randn(len(X_train))
    
    beta_ols_trial = fit_ols(X_train, y_noisy)
    beta_ridge_trial = fit_ridge(X_train, y_noisy, lambda_reg=1.0)
    
    betas_ols.append(beta_ols_trial[1:])  # Exclude intercept
    betas_ridge.append(beta_ridge_trial[1:])

betas_ols = np.array(betas_ols)
betas_ridge = np.array(betas_ridge)

# Compute standard deviation across trials (coefficient instability)
std_ols = betas_ols.std(axis=0)
std_ridge = betas_ridge.std(axis=0)

print("Coefficient Standard Deviation (across 50 noise realizations):")
print("\nOLS:")
print(std_ols)
print(f"Mean std: {std_ols.mean():.3f}")

print("\nRidge (λ=1.0):")
print(std_ridge)
print(f"Mean std: {std_ridge.mean():.3f}")

print(f"\n→ Ridge reduces coefficient variance by {100*(1 - std_ridge.mean()/std_ols.mean()):.0f}%")

In [ ]:
# Visualize coefficient stability
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# OLS
ax = axes[0]
for i in range(n_features):
    ax.scatter([i]*n_trials, betas_ols[:, i], alpha=0.3, s=30)
ax.errorbar(range(n_features), betas_ols.mean(axis=0), yerr=std_ols, 
            fmt='ro', markersize=10, capsize=5, linewidth=2, label='Mean ± Std')
ax.axhline(0, color='gray', linestyle='--', linewidth=1)
ax.set_xlabel('Feature Index', fontsize=12)
ax.set_ylabel('Coefficient Value', fontsize=12)
ax.set_title('OLS: High Variance (Unstable)', fontsize=13, fontweight='bold')
ax.set_xticks(range(n_features))
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# Ridge
ax = axes[1]
for i in range(n_features):
    ax.scatter([i]*n_trials, betas_ridge[:, i], alpha=0.3, s=30)
ax.errorbar(range(n_features), betas_ridge.mean(axis=0), yerr=std_ridge, 
            fmt='go', markersize=10, capsize=5, linewidth=2, label='Mean ± Std')
ax.axhline(0, color='gray', linestyle='--', linewidth=1)
ax.set_xlabel('Feature Index', fontsize=12)
ax.set_ylabel('Coefficient Value', fontsize=12)
ax.set_title('Ridge (λ=1.0): Low Variance (Stable)', fontsize=13, fontweight='bold')
ax.set_xticks(range(n_features))
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(REPORTS_DIR / '03_coefficient_stability.png', dpi=150, bbox_inches='tight')
print(f"✓ Saved: {REPORTS_DIR / '03_coefficient_stability.png'}")
plt.show()

**Key Result**: Ridge coefficients are much more stable (lower variance) across different noise realizations!

---

## 7. Regularization Path: Coefficients vs λ

In [ ]:
# Sweep over λ values
lambdas = np.logspace(-3, 2, 50)  # 0.001 to 100
coef_paths = []
train_errors = []
test_errors = []
condition_numbers = []

for lam in lambdas:
    beta = fit_ridge(X_train, y_train, lambda_reg=lam)
    coef_paths.append(beta[1:])  # Exclude intercept
    
    y_pred_train = predict(X_train, beta)
    y_pred_test = predict(X_test, beta)
    train_errors.append(mse(y_train, y_pred_train))
    test_errors.append(mse(y_test, y_pred_test))
    
    # Condition number of regularized system
    XtX = X_train.T @ X_train
    XtX_reg = XtX + lam * np.eye(XtX.shape[0])
    condition_numbers.append(condition_number(XtX_reg))

coef_paths = np.array(coef_paths)

# Plot regularization path
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Coefficient paths
ax = axes[0, 0]
for i in range(n_features):
    ax.plot(lambdas, coef_paths[:, i], linewidth=2, label=f'Feature {i+1}')
ax.set_xscale('log')
ax.axhline(0, color='gray', linestyle='--', alpha=0.5)
ax.set_xlabel('λ (Regularization Strength)', fontsize=12)
ax.set_ylabel('Coefficient Value', fontsize=12)
ax.set_title('Regularization Path', fontsize=13, fontweight='bold')
ax.legend(fontsize=9, loc='best')
ax.grid(True, alpha=0.3)

# Plot 2: Train vs Test error
ax = axes[0, 1]
ax.plot(lambdas, train_errors, linewidth=2, label='Train MSE', color='blue')
ax.plot(lambdas, test_errors, linewidth=2, label='Test MSE', color='red')
# Mark minimum test error
best_idx = np.argmin(test_errors)
ax.plot(lambdas[best_idx], test_errors[best_idx], 'r*', markersize=15, 
        label=f'Best λ={lambdas[best_idx]:.3f}')
ax.set_xscale('log')
ax.set_xlabel('λ (Regularization Strength)', fontsize=12)
ax.set_ylabel('Mean Squared Error', fontsize=12)
ax.set_title('Bias-Variance Tradeoff', fontsize=13, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# Plot 3: Condition number vs λ
ax = axes[1, 0]
ax.plot(lambdas, condition_numbers, linewidth=2, color='purple')
ax.set_xscale('log')
ax.set_yscale('log')
ax.axhline(10, color='green', linestyle='--', label='κ=10 (well-conditioned)', linewidth=1.5)
ax.set_xlabel('λ (Regularization Strength)', fontsize=12)
ax.set_ylabel('Condition Number κ', fontsize=12)
ax.set_title('Numerical Conditioning vs λ', fontsize=13, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# Plot 4: Coefficient norm vs λ
ax = axes[1, 1]
coef_norms = np.linalg.norm(coef_paths, axis=1)
ax.plot(lambdas, coef_norms, linewidth=2, color='orange')
ax.set_xscale('log')
ax.set_xlabel('λ (Regularization Strength)', fontsize=12)
ax.set_ylabel('||β||₂', fontsize=12)
ax.set_title('Coefficient Shrinkage', fontsize=13, fontweight='bold')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(REPORTS_DIR / '03_ridge_analysis.png', dpi=150, bbox_inches='tight')
print(f"✓ Saved: {REPORTS_DIR / '03_ridge_analysis.png'}")
plt.show()

print(f"\nOptimal λ = {lambdas[best_idx]:.3f} (minimizes test MSE)")
print(f"At optimal λ: condition number = {condition_numbers[best_idx]:.1f}")

**Observations:**
1. **Coefficients shrink** as λ increases (regularization path)
2. **Test error U-shaped**: too little λ → overfit, too much λ → underfit
3. **Condition number decreases** monotonically with λ → numerical stability improves
4. **Optimal λ balances** bias (underfitting) and variance (instability)

---

## 8. Compare to sklearn

In [ ]:
from sklearn.linear_model import Ridge as SklearnRidge

# Our implementation
beta_ours = fit_ridge(X_train, y_train, lambda_reg=1.0)
y_pred_ours = predict(X_test, beta_ours)
mse_ours = mse(y_test, y_pred_ours)

# sklearn
ridge_sklearn = SklearnRidge(alpha=1.0, fit_intercept=True)
ridge_sklearn.fit(X_train, y_train)
y_pred_sklearn = ridge_sklearn.predict(X_test)
mse_sklearn = mse(y_test, y_pred_sklearn)

print("Comparison (λ=1.0):")
print("\nCoefficients:")
print(f"  Ours:    {beta_ours[1:]}")
print(f"  sklearn: {ridge_sklearn.coef_}")
print(f"\nIntercept:")
print(f"  Ours:    {beta_ours[0]:.4f}")
print(f"  sklearn: {ridge_sklearn.intercept_:.4f}")
print(f"\nTest MSE:")
print(f"  Ours:    {mse_ours:.6f}")
print(f"  sklearn: {mse_sklearn:.6f}")
print(f"\nMax coefficient difference: {np.max(np.abs(beta_ours[1:] - ridge_sklearn.coef_)):.2e}")
print("\n✓ Match within numerical precision!")

---

## 9. Key Takeaways

✅ **Ridge = Regularization + Stabilization**:
   - ML view: Prevents overfitting via penalty on ||β||²
   - Numerical view: Fixes ill-conditioned X^T X by shifting eigenvalues

✅ **Condition number improves**: κ(X^T X + λI) ≪ κ(X^T X)
   - Small λ_min in X^T X becomes λ_min + λ → stable inversion
   - Reduces sensitivity to rounding errors

✅ **Coefficient stability**:
   - Ridge coefficients have lower variance across noise realizations
   - Shrinkage toward zero reduces magnitude

✅ **Bias-variance tradeoff**:
   - λ too small → high variance (overfitting)
   - λ too large → high bias (underfitting)
   - Use cross-validation to choose optimal λ

✅ **When to use Ridge**:
   - Correlated features (multicollinearity)
   - More features than samples (p > n)
   - Noisy measurements
   - Always a safe default for linear regression!

---

## 10. Common Pitfalls

❌ **Not standardizing features**: Ridge penalizes all coefficients equally → features with larger scales get less regularization

❌ **Penalizing the intercept**: Should only regularize slope coefficients, not intercept

❌ **Using OLS on ill-conditioned data**: Results are numerically unstable and overfit

❌ **Choosing λ on training set**: Will always prefer λ=0 (OLS); use validation/cross-validation

❌ **Confusing α and λ**: sklearn uses `alpha` parameter, some texts use λ; they're the same thing

---

## 11. Exercises

**Exercise 1**: Implement k-fold cross-validation to find optimal λ automatically.

In [ ]:
# Your code here

**Exercise 2**: Create a dataset with p=20 features but only n=15 samples. Compare OLS (will it even work?) vs Ridge.

In [ ]:
# Your code here

**Exercise 3**: Derive the bias and variance of Ridge estimator. Show that as λ→0, it approaches OLS.

*Your derivation here*

**Exercise 4**: Implement Lasso (L1 regularization) from scratch using coordinate descent. Compare coefficient patterns to Ridge.

In [ ]:
# Your code here

**Exercise 5**: Add Gaussian noise with increasing variance to y. Plot test MSE vs noise level for OLS and Ridge. At what noise level does Ridge become clearly better?

In [ ]:
# Your code here

---

## Solutions

<details>
<summary><b>Exercise 1 Solution</b></summary>

```python
from sklearn.model_selection import KFold

def cross_validate_ridge(X, y, lambdas, n_folds=5):
    """Find optimal lambda via k-fold CV."""
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
    cv_errors = {lam: [] for lam in lambdas}
    
    for train_idx, val_idx in kf.split(X):
        X_train_fold, X_val_fold = X[train_idx], X[val_idx]
        y_train_fold, y_val_fold = y[train_idx], y[val_idx]
        
        for lam in lambdas:
            beta = fit_ridge(X_train_fold, y_train_fold, lambda_reg=lam)
            y_pred = predict(X_val_fold, beta)
            cv_errors[lam].append(mse(y_val_fold, y_pred))
    
    # Average across folds
    mean_errors = {lam: np.mean(errs) for lam, errs in cv_errors.items()}
    best_lambda = min(mean_errors, key=mean_errors.get)
    
    return best_lambda, mean_errors

lambdas_cv = np.logspace(-2, 1, 20)
best_lam, cv_errs = cross_validate_ridge(X_train, y_train, lambdas_cv, n_folds=5)
print(f"Optimal λ via 5-fold CV: {best_lam:.3f}")

# Plot CV curve
plt.figure(figsize=(8, 5))
plt.plot(list(cv_errs.keys()), list(cv_errs.values()), 'o-', linewidth=2)
plt.axvline(best_lam, color='red', linestyle='--', label=f'Best λ={best_lam:.3f}')
plt.xscale('log')
plt.xlabel('λ')
plt.ylabel('CV Error')
plt.title('Cross-Validation Curve')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()
```
</details>

<details>
<summary><b>Exercise 2 Solution</b></summary>

```python
# p > n case
np.random.seed(123)
n, p = 15, 20
X_wide = np.random.randn(n, p)
true_beta_wide = np.random.randn(p)
true_beta_wide[10:] = 0  # Only first 10 features matter
y_wide = X_wide @ true_beta_wide + 0.1 * np.random.randn(n)

# Try OLS - will it fail?
try:
    X_aug = np.column_stack([np.ones(n), X_wide])
    XtX = X_aug.T @ X_aug
    print(f"X^T X shape: {XtX.shape}")
    print(f"Rank of X^T X: {np.linalg.matrix_rank(XtX)}")
    print(f"Should be: {XtX.shape[0]}")
    beta_ols_wide = np.linalg.solve(XtX, X_aug.T @ y_wide)
    print(f"OLS succeeded! But coefficients: {np.linalg.norm(beta_ols_wide):.1f}")
except np.linalg.LinAlgError:
    print("OLS failed: Singular matrix (expected when p > n)")

# Ridge works fine
beta_ridge_wide = fit_ridge(X_wide, y_wide, lambda_reg=1.0)
print(f"\nRidge coefficient norm: {np.linalg.norm(beta_ridge_wide[1:]):.2f}")
print("Ridge handles p > n gracefully!")
```
</details>

---

## Summary Report

In [ ]:
report = f"""
RIDGE REGRESSION AS NUMERICAL STABILIZER
{'='*70}

DATA CHARACTERISTICS:
  Samples: {n_samples}
  Features: {n_features}
  Condition number (X^T X): {kappa:.1f} (ill-conditioned)
  Feature correlations: High (ρ > 0.9)

COEFFICIENT STABILITY (50 noise realizations):
  OLS mean std:   {std_ols.mean():.3f}
  Ridge mean std: {std_ridge.mean():.3f}
  Improvement:    {100*(1 - std_ridge.mean()/std_ols.mean()):.0f}% reduction in variance

OPTIMAL REGULARIZATION:
  Best λ (minimizes test MSE): {lambdas[best_idx]:.3f}
  At optimal λ:
    - Condition number: {condition_numbers[best_idx]:.1f}
    - Test MSE: {test_errors[best_idx]:.4f}
    - Coefficient norm: {coef_norms[best_idx]:.2f}

KEY FINDINGS:
  1. Ridge significantly stabilizes coefficients in ill-conditioned problems
  2. Condition number improves monotonically with λ
  3. Optimal λ balances bias (underfitting) vs variance (overfitting)
  4. Our implementation matches sklearn Ridge exactly

PRACTICAL RECOMMENDATIONS:
  - Always standardize features before Ridge regression
  - Use cross-validation to choose λ
  - For correlated features (ρ > 0.7), Ridge is essential
  - When p ≈ n or p > n, OLS fails but Ridge works

Plots saved in: {REPORTS_DIR}/
"""

print(report)

with open(REPORTS_DIR / '03_ridge_regression_report.txt', 'w') as f:
    f.write(report)

print(f"\n✓ Report saved: {REPORTS_DIR / '03_ridge_regression_report.txt'}")